# Módulo 2 - Desafio

In [2]:
# Importação de bibliotecas
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import mysql.connector
import requests

ModuleNotFoundError: No module named 'mysql'

In [3]:
# API para coletar os dados
response = requests.get("https://api-coleta-dados.herokuapp.com/api/pessoas")

NameError: name 'requests' is not defined

In [ ]:
# Imprimir status da requisição (200)
print(response.status_code)

In [ ]:
# Imprimir retorno da API
print(response.content)

In [ ]:
# Serializar o conteúdo binário da API 
dados = json.loads(response.content)

In [ ]:
for pessoa in dados:
    print(pessoa)

In [ ]:
mydb = mysql.connector.connect(
host = 'localhost',
user = 'root',
passowrd = 'igti',
database = 'desafio_modulo2'
)

In [ ]:
mycursor = mydb.cursor()

In [ ]:
# Inserir estados
for pessoa in dados:
    sigla_uf = pessoa['estado']
    
    query = "INSERT INTO TB ESTADO (sigla) SELECT * FROM(SELECT'%s') AS tmp" % sigla_uf
    query += "WHERE NOT EXISTS (SELECT sigla FROM TB_ESTADO WHERE sigla = '%s') LIMIT 1;" % sigla_uf
    mycursor.execute(query)
    
    # Confirmação da inserção
    mydb.commit()

In [ ]:
print(mycursor.rowcount, "registro(s) inserido(s).")

In [ ]:
# Inserir cidades
for pessoa in dados:
    cidade = pessoa['cidade']
    
    query = "INSERT INTO TB_CIDADE (nome,id_estado) (SELECT * FROM (SELECT '{}', (SELECT id FROM TB_ESTADO " \
            "WHERE SIGLA = '{}')) AS tmp ".format(cidade, pessoa['estado'])
    query += "WHERE NOT EXISTS (SELECT nome FROM TB_CIDADE WHERE nome = '{}')LIMI 1);".format(cidade)
    mycursor.execute(query)
    mydb.commit()

In [ ]:
print(mycursor.rowcount, "registro(s) inserido(s).")

In [ ]:
# Inserir cores
for pessoa in dados:
    cor = pessoa['cor']
    
    query = "INSERT INTO TB_COR(cor) SELECT * FROM (SELECT'{}') AS tmp".format(cor))"
    query += "WHERE NOT EXISTS (select cor FROM TB_COR WHERE cor = '{}') LIMIT 1;".format(cor)
    mycursor.execute(query)
    mydb.commit()

In [ ]:
print(mycursor.rowcount, "registro(s) inserido(s)")

In [ ]:
# Inserir tipo sanguíneo
for pessoa in dados:
    tipo_sanguineo = pessoa['tipo_sanguineo']
    query = "INSERT INTO TB_TIPOSANGUINEO(tipo) SELECT * FROM (SELECT '{}') AS tmp". format(tipo_sanguineo)
    query += "WHERE NOT EXISTS (SELECT tipo FROM TB_TIPOSANGUINEO WHERE tipo = '{}') LIMIT 1;".format(tipo_sanguineo)
    mycursor.execute(query)
    mydb.commit()

In [ ]:
print(mycursor.rowcount, "registro(s) inserido(s).")

In [ ]:
for pessoa in dados:
    if pessoa['idade'] is None:
        idade = relativedelta(datetime.now(), datetime.strptime(pessoa['data_nasc'], "%d/%m/%Y"))
        pessoa['idade'] = idade.years
    
    if len(pessoa['data_nasc']) < 10:
        diff_date = datetime.now() - relativedelta(years=pessoa['idade'])
        ano_nasc = diff_date.year
        if datetime.strptime(pessoa['data_nasc'] + '/' + str(datetime.now().year), "%d/%m/%Y") > datetime.now():
            ano_nasc = ano_nasc - 1
        pessoa['data_nasc'] = pessoa['data_nasc'] + str(ano_nasc)
    
    query = "INSERT INTO TB_PESSOA (nome,idade,data_nasc,sexo,signo,altura,peso,id_cidade,id_cor,id_tiposanguineo) " \
            "VALUES " \
            "('{}','{}',str_to_date('{}','%d/%m/%Y'),'{}','{}','{}','{}', (SELECT id FROM TB_CIDADE " \
            "WHERE nome = '{}'), (SELECT id FROM TB_COR WHERE cor = '{}'), (SELECT id FROM TB_TIPOSANGUINEO WHERE " \
            "tipo = '{}')); ".format(pessoa['nome'], pessoa['idade'], pessoa['data_nasc'], pessoa['sexo'][0:1],
                                     pessoa['signo'], pessoa['altura'].replace(',', '.'), pessoa['peso'],
                                     pessoa['cidade'],
                                     pessoa['cor'], pessoa['tipo_sanguineo'])
    mycursor.execute(query)
    mydb.commit()

In [ ]:
print(mycursor.rowcount, "registro(s) inserido(s).")